In [ ]:
# user_program.py

import requests
import uuid
import logging
import time
import psutil
import os

class UserProgram:
    def __init__(self, pep_url):
        self.pep_url = pep_url  # URL of the PEP
        self.user_id = 'user'
        self.ip_address = '192.52.36.185'
        self.username = 'user'
        self.password = '123'
        self.setup_logging()

    def setup_logging(self):
        # Configure the logging module
        logging.basicConfig(
            filename='user_program.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )

    def start(self):
        while True:
            print("\nSelect an option:")
            print("1. Read data from iot_device1")
            print("2. Read data from iot_device2")
            print("3. Read data from server")
            print("4. Write data to server")
            print("q. Quit")
            choice = input("Enter your choice: ").strip()

            if choice == '1':
                self.request_device_data('iot_device1')
            elif choice == '2':
                self.request_device_data('iot_device2')
            elif choice == '3':
                self.read_server_data()
            elif choice == '4':
                self.write_server_data()
            elif choice.lower() == 'q':
                print("Exiting user program.")
                break
            else:
                print("Invalid choice.")

    def request_device_data(self, device_id):
        resource_id = f'{device_id}_data'
        request_data = {
            'username': self.username,
            'password': self.password,
            'ip_address': self.ip_address
        }
        print(f"Requesting data from '{device_id}'...")
        self.send_request_to_pep(resource_id, request_data)

    def read_server_data(self):
        resource_id = 'server_data'
        request_data = {
            'username': self.username,
            'password': self.password,
            'ip_address': self.ip_address,
            'action': 'read'
        }
        print("Requesting to read data from server...")
        self.send_request_to_pep(resource_id, request_data)

    def write_server_data(self):
        data_to_write = input("Enter data to write to the server: ")
        resource_id = 'server_data'
        request_data = {
            'username': self.username,
            'password': self.password,
            'ip_address': self.ip_address,
            'action': 'write',
            'data': data_to_write
        }
        print("Requesting to write data to server...")
        self.send_request_to_pep(resource_id, request_data)

    def send_request_to_pep(self, resource_id, request_data):
        request_id = str(uuid.uuid4())

        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        payload = {
            'entity_id': self.user_id,
            'resource_id': resource_id,
            'request_data': request_data,
            'request_id': request_id
        }

        try:
            response = requests.post(f'{self.pep_url}/handle_request', json=payload)
            response.raise_for_status()
            data = response.json()
            access_decision = data.get('access_decision', 'deny')
            print(f"Access decision: {access_decision}")

            # End CPU and timing measurements
            end_time = time.time()
            cpu_times_end = process.cpu_times()

            # Calculate CPU time and processing time
            user_cpu_time = cpu_times_end.user - cpu_times_start.user
            system_cpu_time = cpu_times_end.system - cpu_times_start.system
            total_cpu_time = user_cpu_time + system_cpu_time
            processing_time = end_time - start_time

            # Log the performance data
            logging.info(
                f"Request ID: {request_id} | UserProgram: Sent request to PEP | Resource: {resource_id} | "
                f"Access Decision: {access_decision} | Processing Time: {processing_time:.6f}s | "
                f"CPU Time: User={user_cpu_time:.6f}s System={system_cpu_time:.6f}s "
                f"Total={total_cpu_time:.6f}s"
            )

        except requests.exceptions.RequestException as e:
            logging.error(f"Request ID: {request_id} | UserProgram Error: Failed to send request to PEP: {e}")
            print(f"Error: Could not send request to PEP. Details logged.")
        except Exception as e:
            logging.error(f"Request ID: {request_id} | UserProgram Error: {e}")
            print(f"An unexpected error occurred. Details logged.")

if __name__ == '__main__':
    # Replace with the actual PEP URL
    pep_url = 'http://192.52.34.188:5000'  # Replace with actual IP or hostname

    user_program = UserProgram(pep_url)
    user_program.start()